# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Version Info
- 모델 학습 방법 : Efficientnet_b4
- Augmentation + 하이퍼파라미터 + 가중치 + 스케줄러 + weight decay + TTA + 홀드아웃 + 오프라인 증강

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

In [1]:
# 구글 드라이브 마운트, Colab을 이용하지 않는다면 패스해도 됩니다.
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
drive.mount('/content/drive')

Mounted at /gdrive
Mounted at /content/drive


In [2]:
# 구글 드라이브에 업로드된 대회 데이터를 압축 해제하고 로컬에 저장합니다.
!tar -xvf drive/MyDrive/datasets_fin.tar > /dev/null

In [11]:
# 필요한 라이브러리를 설치합니다.
!pip install timm

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [2]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [3]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [5]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [6]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = 'datasets_fin/'

# model config
model_name = 'efficientnet_b4' # 'efficientnet_b1' #'resnet50' 'resnet34' # , ...

# training config - 이 부분을 수정하여 성능을 높일 수 있음 
img_size = 224
LR = 1e-4
EPOCHS = 100
BATCH_SIZE = 8
num_workers = 2 # 속도 향상을 위해 조정
EARLY_STOPPING_PATIENCE = 10 # 5 에포크 동안 점수 향상 없으면 중단


## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [7]:
import cv2
import pandas as pd
from tqdm import tqdm
import albumentations as A
import os
from sklearn.model_selection import train_test_split

# 원본 CSV 불러오기
all_df = pd.read_csv("datasets_fin/train.csv")

# [✅ 핵심] 1570개를 8:2 비율로 '학습용'과 '미니 리더보드용'으로 분리
# stratify=all_df['target'] : 17개 클래스 비율을 유지하며 섞어줌 (Macro F1 문제 해결!)
train_df, holdout_df = train_test_split(
    all_df, 
    test_size=0.2,    # 20%를 미니 리더보드(holdout)로 사용
    random_state=42,  # 항상 동일하게 분리
    stratify=all_df['target'] 
)

# 학습은 80% 데이터로만 진행 (1570 * 0.8 = 약 1256개)
df = train_df.reset_index(drop=True) 

print(f"총 데이터: {len(all_df)}, K-Fold 학습용: {len(df)}, 미니 리더보드용: {len(holdout_df)}")

# holdout_df (약 314개)는 '미니 리더보드'의 정답지 역할을 하며,
# 절대 K-Fold 학습 과정에서 사용되지 않습니다.

총 데이터: 1570, K-Fold 학습용: 1256, 미니 리더보드용: 314


* 오프라인 데이터 증강 수행

In [58]:
print("오프라인 데이터 증강을 시작합니다...")

# 원본 폴더와 증강 이미지를 저장할 폴더
input_dir = "datasets_fin/train"
output_dir = "datasets_fin/train_aug"
os.makedirs(output_dir, exist_ok=True)

light_non_paper_augment = A.Compose([
    # 1. 극단적인 원근 왜곡 (테스트 셋의 핵심 특징)
    A.Perspective(scale=(0.05, 0.1), pad_mode=cv2.BORDER_CONSTANT, p=0.5),
    
    # 2. 회전, 기울임, 크기 조절
    A.Affine(
        rotate=(-45, 45),       # 회전 범위
        shear=(-15, 15),       # 기울임
        scale=(0.8, 1.2),      # 확대/축소
        p=0.8
    ),
    
    # 3. 각종 블러 (모션/초점/노이즈)
    A.OneOf([
        A.GaussianBlur(blur_limit=(3, 7), p=0.5),
        A.MotionBlur(blur_limit=(3, 7), p=0.5), # 모션 블러
        A.MedianBlur(blur_limit=5, p=0.5)
    ], p=0.7),
    
    # 4. 어두운 조명, 그림자, 품질 저하
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.4, contrast_limit=0.4, p=0.8),
        A.RandomGamma(gamma_limit=(50, 150), p=0.8) # 극단적 감마
    ], p=0.9),
    
    # 5. 노이즈 및 JPEG 압축 흉내
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
    A.ImageCompression(quality_lower=50, quality_upper=80, p=0.5), # JPEG 아티팩트
    
    # 6. 문서 잘리게 (V3와 동일)
    A.CoarseDropout(
        max_holes=8, 
        max_height=int(img_size * 0.2), 
        max_width=int(img_size * 0.2), 
        min_holes=1, 
        fill_value=0, 
        p=0.7
    ),

    # 7. 좌우반전
    A.HorizontalFlip(p=0.3)
])

strong_paper_augment = A.Compose([
    A.OneOf([
        A.Perspective(scale=(0.05, 0.1), pad_mode=cv2.BORDER_CONSTANT, p=1.0),
        A.GridDistortion(num_steps=5, distort_limit=0.3, p=1.0), 
    ], p=0.8),
    A.Affine(rotate=(-45, 45), shear=(-15, 15), scale=(0.8, 1.2), p=0.8),
    A.OneOf([
        A.GaussianBlur(blur_limit=(3, 15), p=0.5), 
        A.MotionBlur(blur_limit=(3, 15), p=0.5), 
        A.MedianBlur(blur_limit=11, p=0.5), 
        A.Blur(blur_limit=(3, 11), p=0.5), 
        A.Downscale(scale_min=0.3, scale_max=0.5, p=0.5) 
    ], p=0.9), 
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.4, contrast_limit=0.4, p=0.8),
        A.RandomGamma(gamma_limit=(50, 150), p=0.8)
    ], p=0.9),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
    A.ImageCompression(quality_lower=50, quality_upper=80, p=0.5),
    A.CoarseDropout(max_holes=8, max_height=int(img_size * 0.2), max_width=int(img_size * 0.2), 
                    min_holes=1, fill_value=0, p=0.7),
    A.HorizontalFlip(p=0.5) 
])


# -----------------------------------------------------------------
# [✅ V5 전략] '조건부' 오프라인 증강 루프
# -----------------------------------------------------------------

# (사용자님 분류 기준) '종이' 타겟 리스트
PAPER_TARGETS = {1, 3, 4, 6, 7, 11, 12, 14, 15, 0, 10, 13}

aug_data = [] # 증강된 파일 목록을 저장할 리스트

# (df는 Cell 11에서 1256개로 정의됨)
for idx, row in tqdm(df.iterrows(), total=len(df)):
    img_name = row['ID']
    label = row['target']
    img_path = os.path.join(input_dir, img_name)
    
    image = cv2.imread(img_path)
    if image is None:
        print(f"경고: {img_path} 파일을 읽을 수 없습니다. 건너뜁니다.")
        continue
    
    # 1. 원본 이미지를 새 폴더에 그대로 저장
    cv2.imwrite(os.path.join(output_dir, img_name), image)
    aug_data.append({"ID": img_name, "target": label})

    # [✅ V5 핵심 로직] 라벨에 따라 사용할 증강을 선택
    if label in PAPER_TARGETS:
        augment_to_use = strong_paper_augment # '종이'용 강력한 증강
    else:
        augment_to_use = light_non_paper_augment # '종이 아님'용 가벼운 증강
        
    # 2. 증강된 이미지 4개 생성 및 저장
    for i in range(4): # 4번 반복
        # [✅ V5] 선택된 증강을 적용
        augmented = augment_to_use(image=image)["image"] 
        new_name = f"{os.path.splitext(img_name)[0]}_aug{i+1}.jpg"
        cv2.imwrite(os.path.join(output_dir, new_name), augmented)
        aug_data.append({"ID": new_name, "target": label})

# 새 CSV 파일 (aug_train.csv) 저장
aug_df = pd.DataFrame(aug_data)
aug_df.to_csv("datasets_fin/aug_train.csv", index=False) 

print(f"✅ [V5 - 조건부 증강] 데이터 증강 및 CSV 생성 완료! (총 {len(aug_df)}개 이미지)")

/opt/conda/lib/python3.10/site-packages/albumentations/augmentations/transforms.py:1692: UserWarning: Using default interpolation INTER_NEAREST, which is sub-optimal.Please specify interpolation mode for downscale and upscale explicitly.For additional information see this PR https://github.com/albumentations-team/albumentations/pull/584
  warnings.warn(


오프라인 데이터 증강을 시작합니다...


100%|██████████| 1256/1256 [01:50<00:00, 11.33it/s]

✅ [V5 - 조건부 증강] 데이터 증강 및 CSV 생성 완료! (총 6280개 이미지)


In [8]:
# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size),

    # 극심한 회전 및 원근 왜곡 추가
    A.Perspective(scale=(0.05, 0.1), pad_mode=0, p=0.7),
    A.Affine(
        rotate=(-90, 90),       # -90도 ~ +90도 사이로 마구 회전
        shear=(-15, 15),       # 이미지 기울임
        scale=(0.8, 1.2),      # 80%~120% 확대/축소
        translate_percent=0.1, # 상하좌우 이동
        p=0.8
    ),
    # 조명 및 품질 저하 흉내내기
    A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1, p=0.8),
    A.OneOf([
        A.GaussianBlur(blur_limit=(3, 7), p=0.5),
        A.MotionBlur(blur_limit=(3, 7), p=0.5),
    ], p=0.5),
    # 이미지 일부 가리기 (과적합 방지)
    A.CoarseDropout(max_holes=8, max_height=img_size//8, max_width=img_size//8, min_holes=1, p=0.7),
    # 좌우 반전
    # 자동차 사진도 있고, 문서도 뒤집혀 있으므로 HorizontalFlip은 좋은 증강입니다.
    A.HorizontalFlip(p=0.5),

    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [9]:
# Validation 함수
def validate_one_epoch(loader, model, loss_fn, device):
    model.eval() # 모델을 평가 모드로 설정
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad(): # 기울기 계산 비활성화
        pbar = tqdm(loader, desc="Valid")
        for image, targets in pbar:
            image = image.to(device)
            targets = targets.to(device)
            
            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())
            pbar.set_description(f"Valid Loss: {loss.item():.4f}")

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro') # Macro F1

    ret = {
        "val_loss": val_loss,
        "val_acc": val_acc,
        "val_f1": val_f1,
    }
    return ret

In [10]:
class ImageDataset(Dataset):
    def __init__(self, df, path, transform=None, target_col='target'):
        self.df = df 
        self.path = path
        self.transform = transform
        self.target_col = target_col

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name = self.df.iloc[idx]['ID']
        target = self.df.iloc[idx][self.target_col]
        
        # 이미지 열기
        img_path = os.path.join(self.path, name)
        try:
            img = np.array(Image.open(img_path))
        except FileNotFoundError:
            print(f"Error: Image file not found at {img_path}")
            # 또는 기본 이미지 반환
            return torch.randn(3, img_size, img_size), 0 
            
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [62]:
from sklearn.model_selection import StratifiedKFold

# Dataset 및 DataLoader 정의
trn_dataset = ImageDataset(
    aug_df,
    "datasets_fin/train_aug/",
    transform=trn_transform,
    target_col='target'
)
val_dataset = ImageDataset(
    holdout_df, 
    "datasets_fin/train/", 
    transform=val_transform,
    target_col='target'
)

# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)

# Loss 가중치 계산
current_num_classes = 17
weights = torch.ones(current_num_classes, dtype=torch.float32).to(device)

class_counts = train_df['target'].value_counts()

if not class_counts.empty:
    weight_values = 1. / torch.tensor(class_counts.values, dtype=torch.float32).to(device)
    weights.scatter_(0, torch.tensor(class_counts.index, dtype=torch.long).to(device), weight_values)

weights = (weights / weights.sum()) * current_num_classes 
loss_fn = nn.CrossEntropyLoss(weight=weights)

# Optimizer, Scheduler
optimizer = Adam(model.parameters(), lr=LR, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)

# Early Stopping을 위한 변수
best_val_f1 = 0.0
patience_counter = 0

print(f"--- '미니 리더보드' 학습 시작 ---")
print(f"학습 데이터: {len(trn_dataset)}개 (증강됨)")
print(f"검증 데이터: {len(val_dataset)}개 (깨끗한 원본)")

for epoch in range(EPOCHS):
    start_time = time.time()

    train_ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    
    # [핵심] 314개의 깨끗한 홀드아웃 셋으로 검증
    val_ret = validate_one_epoch(val_loader, model, loss_fn, device)
    
    train_ret['epoch'] = epoch
    elapsed = time.time() - start_time
    scheduler.step()
    
    print(f"Epoch {epoch+1}/{EPOCHS} [{elapsed:.0f}s] - "
          f"Train F1: {train_ret['train_f1']:.4f}, "
          f"Valid F1 (Holdout): {val_ret['val_f1']:.4f}, " # <- 이게 진짜 점수
          f"Valid Loss: {val_ret['val_loss']:.4f}")

    # Early Stopping 및 모델 저장 로직
    if val_ret['val_f1'] > best_val_f1:
        best_val_f1 = val_ret['val_f1']
        
        # [✅ 수정됨] 저장 경로에서 {fold} 변수 제거
        torch.save(model.state_dict(), f"model/best_eff_model.pth") 
        print(f"⭐️ Best F1 Score updated to {best_val_f1:.4f}. Model saved!")
        patience_counter = 0 # 초기화
    else:
        patience_counter += 1
    
    if patience_counter >= EARLY_STOPPING_PATIENCE:
        print(f"Early stopping at epoch {epoch+1} as F1 score did not improve for {EARLY_STOPPING_PATIENCE} epochs.")
        break

print(f"--- 학습 완료 ---")
print(f"최종 '미니 리더보드' F1 점수: {best_val_f1:.4f}")
        

--- '미니 리더보드' 학습 시작 ---
학습 데이터: 6280개 (증강됨)
검증 데이터: 314개 (깨끗한 원본)


Valid Loss: 0.2868: 100%|██████████| 40/40 [00:01<00:00, 38.17it/s]


Epoch 1/100 [41s] - Train F1: 0.4483, Valid F1 (Holdout): 0.4996, Valid Loss: 1.4133
⭐️ Best F1 Score updated to 0.4996. Model saved!


Valid Loss: 0.3752: 100%|██████████| 40/40 [00:00<00:00, 50.91it/s]


Epoch 2/100 [40s] - Train F1: 0.7288, Valid F1 (Holdout): 0.6619, Valid Loss: 1.0405
⭐️ Best F1 Score updated to 0.6619. Model saved!


...

Epoch 26/100 [40s] - Train F1: 0.9727, Valid F1 (Holdout): 0.9124, Valid Loss: 0.3664


Valid Loss: 0.0006: 100%|██████████| 40/40 [00:00<00:00, 49.31it/s]


Epoch 27/100 [39s] - Train F1: 0.9766, Valid F1 (Holdout): 0.8878, Valid Loss: 0.4911


Valid Loss: 0.2356: 100%|██████████| 40/40 [00:00<00:00, 48.17it/s]

Epoch 28/100 [40s] - Train F1: 0.9766, Valid F1 (Holdout): 0.8893, Valid Loss: 0.5308
Early stopping at epoch 28 as F1 score did not improve for 10 epochs.
--- 학습 완료 ---
최종 '미니 리더보드' F1 점수: 0.9383


# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [ ]:
import numpy as np
import pandas as pd
import torch
import timm
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm

# --- TTA 적용 '미니 리더보드' 점수 계산 시작 ---
print("--- TTA 적용 '미니 리더보드' 점수 계산 시작 ---")

# (주의: Cell 54에서 생성된 'holdout_df' 변수가 살아있어야 합니다)
if 'holdout_df' not in globals():
    print("오류: 'holdout_df'가 없습니다. Load Data 섹터를 다시 실행해야 할 수 있습니다.")
else:
    print(f"홀드아웃 데이터(holdout_df) 로드 중... (총 {len(holdout_df)}개)")

    # 1. TTA용 데이터로더 생성
    # [✅ 핵심] 'transform=tst_transform'을 사용합니다!
    holdout_tta_dataset = ImageDataset(
        holdout_df,               
        "datasets_fin/train/",    # 깨끗한 원본 이미지가 있는 'train' 폴더
        transform=tst_transform,  # TTA가 포함된 'tst_transform' 사용
        target_col='target'
    )

    holdout_tta_loader = DataLoader(
        holdout_tta_dataset,
        batch_size=BATCH_SIZE * 2, # 추론은 배치 2배로
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    # 2. 저장된 베스트 모델 불러오기
    model = timm.create_model(model_name, pretrained=False, num_classes=17).to(device)
    model_path = "model/best_eff_model.pth" # 학습에서 저장된 경로
    model.load_state_dict(torch.load(model_path))
    model.eval()
    print(f"모델 {model_path} 로드 완료.")

    # 3. TTA 추론 실행 (Inference 섹션 코드 재활용)
    tta_preds_list = []
    with torch.no_grad():
        for image, _ in tqdm(holdout_tta_loader, desc="TTA 추론 중"):
            image = image.to(device)
            
            # TTA 0: 원본 예측
            pred_orig = model(image)

            # TTA 1: 좌우 반전
            image_hflip = torch.flip(image, dims=[-1]) 
            pred_hflip = model(image_hflip)
            
            # TTA 2: 90도 회전
            image_rot90 = torch.rot90(image, k=1, dims=[-2, -1])
            pred_rot90 = model(image_rot90)

            # 3개 예측의 확률(softmax)을 평균냄
            avg_batch_pred = (pred_orig.softmax(dim=1) + 
                              pred_hflip.softmax(dim=1) + 
                              pred_rot90.softmax(dim=1)) / 3.0
            
            tta_preds_list.append(avg_batch_pred.cpu().numpy())
    
    # (314, 17) 크기의 배열로 합치기
    final_tta_preds_array = np.concatenate(tta_preds_list, axis=0)

    # 4. TTA 적용 점수 계산
    # 최종 예측 라벨 (가장 확률 높은 것)
    final_tta_labels = np.argmax(final_tta_preds_array, axis=1)
    
    # 실제 정답 라벨
    true_labels = holdout_df['target'].values
    
    # Macro F1 스코어 계산
    tta_f1_score = f1_score(true_labels, final_tta_labels, average='macro')
    tta_acc_score = accuracy_score(true_labels, final_tta_labels)

    print("\n--- TTA 적용 '미니 리더보드' 최종 점수 ---")
    print(f"✅ TTA 적용 Macro F1: {tta_f1_score:.4f}")
    print(f"   (TTA 적용 Accuracy: {tta_acc_score:.4f})")

    # 홀드아웃 예측 확률 저장
    # np.save("npy/effb4_v5_holdout_preds.npy", final_tta_preds_array)

--- TTA 적용 '미니 리더보드' 점수 계산 시작 ---
홀드아웃 데이터(holdout_df) 로드 중... (총 314개)
모델 model/best_eff_model.pth 로드 완료.


TTA 추론 중: 100%|██████████| 20/20 [00:21<00:00,  1.09s/it]


--- TTA 적용 '미니 리더보드' 최종 점수 ---
✅ TTA 적용 Macro F1: 0.9216
   (TTA 적용 Accuracy: 0.9236)


In [65]:
tst_df = pd.read_csv("datasets_fin/sample_submission.csv")
tst_dataset = ImageDataset(
    tst_df,
    "datasets_fin/test/",
    transform=tst_transform  # Test 셋에는 증강이 없는 val_transform 사용
)

tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE * 2,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

# 2. 저장된 베스트 모델 불러오기
model = timm.create_model(model_name, pretrained=False, num_classes=17).to(device)
model_path = "model/best_eff_model.pth" # 학습에서 저장된 경로
model.load_state_dict(torch.load(model_path))
model.eval()
print(f"모델 {model_path} 로드 완료.")

# 3. TTA 추론 실행 (Inference 섹션 코드 재활용)
tta_preds_list = []
with torch.no_grad():
    for image, _ in tqdm(tst_loader, desc="TTA 추론 중"):
        image = image.to(device)
        
        # TTA 0: 원본 예측
        pred_orig = model(image)

        # TTA 1: 좌우 반전
        image_hflip = torch.flip(image, dims=[-1]) 
        pred_hflip = model(image_hflip)
        
        # TTA 2: 90도 회전
        image_rot90 = torch.rot90(image, k=1, dims=[-2, -1])
        pred_rot90 = model(image_rot90)

        # 3개 예측의 확률(softmax)을 평균냄
        avg_batch_pred = (pred_orig.softmax(dim=1) + 
                            pred_hflip.softmax(dim=1) + 
                            pred_rot90.softmax(dim=1)) / 3.0
        
        tta_preds_list.append(avg_batch_pred.cpu().numpy())

# 4. 최종 예측 라벨 생성
final_tta_preds_array = np.concatenate(tta_preds_list, axis=0)

np.save("npy/effnet_final_preds.npy", final_tta_preds_array) 
print("eff 평균 확률 저장 완료!")

final_tta_labels = np.argmax(final_tta_preds_array, axis=1)

# 5. [✅ 수정됨] F1 스코어 계산 부분 ***완전 삭제***
# (우리는 '진짜 테스트 데이터'의 정답을 모르므로 계산 불가능)
print("추론 완료. 제출 파일 생성 중...")

# 6. [✅ 수정됨] 제출 파일 생성
pred_df = tst_df.copy()
# [중요!] '가짜 정답'(true_labels)이 아닌, '모델의 예측'(final_tta_labels)을 넣어야 함
pred_df['target'] = final_tta_labels

모델 model/best_eff_model.pth 로드 완료.


TTA 추론 중: 100%|██████████| 197/197 [00:09<00:00, 20.42it/s]

eff 평균 확률 저장 완료!
추론 완료. 제출 파일 생성 중...


In [12]:
assert (tst_df['ID'] == pred_df['ID']).all()

In [ ]:
pred_df.to_csv("submission/pred12.csv", index=False)
print("Submission file created!")

Submission file created!
